#### Threat analysis possed by top 50 delegates of ENS

**FAQ:** Why only top 50? <br>
It is not hard-coded. One can provide a list of as many delegates as they want to the script. At the current moment however, top 50 delegates combine to make up just over 68% of the total voting power of ENS DAO. This is sufficient to achieve super-majority. It makes sense to make the cut-off at 50 at the time being.

In [1]:
# Cell 1: environment

import matplotlib.font_manager as fm
import numpy as np
import random as rand
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.pyplot import gca
import os 
import math as mt
from bs4 import BeautifulSoup
from matplotlib.ticker import FormatStrFormatter
import matplotlib.ticker as ticker
import pandas as pd
import requests
import scipy.fftpack
from pprint import pprint
%matplotlib inline

cwd = os.getcwd()
pwd = os.path.abspath(os.path.join(cwd, os.pardir))
fontloc = pwd + '/SF-Mono/SFMono-Regular.otf'
font = fm.FontProperties(fname = fontloc,size = 8); prop = font
proplr = fm.FontProperties(fname = fontloc,size = 12)
prop10 = fm.FontProperties(fname = fontloc,size = 10)

fontlocit = pwd + '/SF-Mono/SFMono-RegularItalic.otf'
fontit = fm.FontProperties(fname = fontlocit,size = 8)

#ticks font
def ticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
        
def ticks3(ax, x, y, z):
    font = fm.FontProperties(fname = fontloc,size = x)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
    font = fm.FontProperties(fname = fontloc,size = y)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)
    font = fm.FontProperties(fname = fontloc,size = z)
    for label in ax.get_zticklabels():
        label.set_fontproperties(font)
        
def xticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_xticklabels():
        label.set_fontproperties(font)
        
def yticks(ax, size):
    font = fm.FontProperties(fname = fontloc,size = size)
    for label in ax.get_yticklabels():
        label.set_fontproperties(font)

@ticker.FuncFormatter
def major_formatter(x, pos):
    label = str("{0:.1f}".format(x)) if x < 0 else str("{0:.2f}".format(x))
    return label

import matplotlib.colors as colors

def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    new_cmap = colors.LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap


#### Utopia

Consider the hypothetical utopic scenario first. In the trivial utopic scenario where the votes are equally distributed among the delegates following a flat power-spectrum, the analytical estimation of attack surface becomes trivial as well. 

In [5]:
%config InlineBackend.figure_format = 'svg'
import itertools

totalVotes = int(np.sum(vote)/(np.sum(perc)*0.01))
print('totalVotes: ' + str(totalVotes))
threshold = 1000000
Total = len(rank)

#N-party attack in Utopia
vote = [totalVotes/Total for i in range(Total)]
perc = [100.0/Total for i in range(Total)]

N = 4; possVec = []
for perps in range(1, Total):
    print('possVec: ' + str(float(np.math.factorial(Total)/np.math.factorial(Total - perps))))
    possVec.append(int(np.math.factorial(Total)/np.math.factorial(Total - perps)))

for perps in range(3,N + 1):
    perms = list(itertools.permutations(rank,perps))
    threat = []; attackers = []
    for val in perms:
      attack = 0; names = ''; ranks = []
      for i in val:
          attack = attack + vote[int(i - 1)]
          #names = names + name[int(i - 1)] + ';'
          ranks.append(rank[int(i - 1)])
      if attack > threshold:
           #attackers.append(names)
           threat.append(ranks)
      attack = 0
      

    print(perps, len(threat), len(perms))




totalVotes: 6847327
possVec: 50.0
possVec: 2450.0
possVec: 117600.0
possVec: 5527200.0
possVec: 254251200.0
possVec: 11441304000.0
possVec: 503417376000.0
possVec: 21646947168000.0
possVec: 909171781056000.0
possVec: 3.7276043023296e+16
possVec: 1.49104172093184e+18
possVec: 5.815062711634176e+19
possVec: 2.209723830420987e+21
possVec: 8.175978172557651e+22
possVec: 2.9433521421207546e+24
possVec: 1.030173249742264e+26
possVec: 3.502589049123698e+27
possVec: 1.1558543862108202e+29
possVec: 3.698734035874625e+30
possVec: 1.1466075511211338e+32
possVec: 3.439822653363401e+33
possVec: 9.975485694753863e+34
possVec: 2.793135994531082e+36
possVec: 7.541467185233921e+37
possVec: 1.9607814681608194e+39
possVec: 4.9019536704020485e+40
possVec: 1.1764688808964917e+42
possVec: 2.705878426061931e+43
possVec: 5.9529325373362475e+44
possVec: 1.250115832840612e+46
possVec: 2.500231665681224e+47
possVec: 4.750440164794326e+48
possVec: 8.550792296629786e+49
possVec: 1.4536346904270637e+51
possVec: 2.3

#### Real World

File [ENS.log](https://github.com/inplco/dao-attacks/blob/master/ENS/ENS.log) contains voting data of top 50 ENS delegates scrapped from [Sybil.org](https://sybil.org). The file header contains four columns: <br>
<code>#</code>rank of the delegate, <br>
<code>#</code>name of delegate, <br>
<code>#</code>voting share in percentage of the delegate, and <br>
<code>#</code>number of votes of the delegate.

In [6]:
# read in data
file_in = cwd + '/ENS.log'
f = open(file_in, 'r')
lines = f.readlines()[1:]
f.close()

rank = []
perc = []
vote = []
name = []

for line in lines:
    if not line.startswith("#"):
        column = line.split()
        rank.append(int(column[0]))
        name.append(str(column[1]))
        perc.append(float(column[2]))
        vote.append(int(column[3]))
            
lines = []

In [7]:
#N-party attack in reality

for perps in range(3,N + 1):
    perms = list(itertools.permutations(rank,perps))
    threat = []; attackers = []
    for val in perms:
      attack = 0; names = ''; ranks = []
      for i in val:
          attack = attack + vote[int(i - 1)]
          #names = names + name[int(i - 1)] + ';'
          ranks.append(rank[int(i - 1)])
      if attack > threshold:
           #attackers.append(names)
           threat.append(ranks)
      attack = 0

    print(perps, len(threat), len(perms))

3 0 117600
4 7248 5527200
